In [69]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices() )
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Current device: {}".format(tf.test.gpu_device_name()))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6454493232708394033
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4937233203
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1182512353633700584
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Num GPUs Available:  1
Current device: /device:GPU:0
